# Chapter 6

## Exercise 1: Logic Definition of Generalization

(a) Show empirically that the information limit of 2 prediction bits per parameter also holds for nearest neighbors.

**Solution**:

In [20]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mutual_info_score
import matplotlib.pyplot as plt


def fit_model(model, X, y, n):
    model.fit(X[:n], y[:n])
    pred_y = model.predict(X)
    
    return np.mean(pred_y == y)


def test_function(n_samples, d):
    X, y = make_classification(n_samples=n_samples, n_features=d, n_informative=d, n_redundant=0, n_classes=2)
    
    for i in range(1, n_samples):
        acc = fit_model(KNeighborsClassifier(n_neighbors=1), X, y, i)
        if acc > 0.999:
            return i
    
    return -1
            
for i in range(1, 5):
    d = 2 ** i
    n_samples = 100
    
    n = []
    for _ in range(10):
        n.append(test_function(n_samples, d))
    
    n_avg = np.mean(n)
    print(f"Dimension: {d}, n: {n_avg}, {n_samples / n_avg}")



Dimension: 2, n: 80.2, 1.2468827930174564
Dimension: 4, n: 72.4, 1.3812154696132595
Dimension: 8, n: 72.2, 1.3850415512465373
Dimension: 16, n: 56.1, 1.7825311942959001


We can see that the number is approaching 2 as the dimension increases.

(b) Extend your experiments to multi-class classification.

**Solution**:

In [24]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mutual_info_score
import matplotlib.pyplot as plt


def fit_model(model, X, y, n):
    model.fit(X[:n], y[:n])
    pred_y = model.predict(X)
    
    return np.mean(pred_y == y)


def test_function(n_samples, d):
    X, y = make_classification(n_samples=n_samples, n_features=d, n_informative=d, n_redundant=0, n_classes=3)
    
    for i in range(1, n_samples):
        acc = fit_model(KNeighborsClassifier(n_neighbors=1), X, y, i)
        if acc > 0.999:
            return i
    
    return -1
            
for i in range(1, 8):
    d = 3 ** i
    n_samples = 100
    
    n = []
    for _ in range(20):
        n.append(test_function(n_samples, d))
    
    n_avg = np.mean(n)
    print(f"Dimension: {d}, n: {n_avg}, {n_samples / n_avg}")



Dimension: 3, n: 75.2, 1.3297872340425532
Dimension: 9, n: 86.8, 1.1520737327188941
Dimension: 27, n: 81.9, 1.2210012210012209
Dimension: 81, n: 78.15, 1.2795905310300704
Dimension: 243, n: 68.0, 1.4705882352941178
Dimension: 729, n: 58.45, 1.7108639863130881
Dimension: 2187, n: 63.2, 1.5822784810126582


We can see that in the 3-class classification problem, we are converging to $3/(3-1) = 1.5$.

## Exercise 2: Finite State Machine Generalization

(a) Implement a program that automatically creates a set of if-then clauses from the training table of a binary dataset of your choice. Implement different strategies to minimize the number of if-then clauses. Document your strategies, the number of resulting conditional clauses, and the accuracy achieved.

**Solution**: 

In [61]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load the Iris dataset
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = (data.target == 1).astype(int)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df[data.feature_names], df['target'], test_size=0.2, random_state=42)

def extract_rules(X, y):
    rules = []
    for column in X.columns:
        values = sorted(X[column].unique())
        for value in values:
            if y[X[column] <= value].mean() > 0.98:
                rules.append((column, '<=', value))
                break
            elif y[X[column] <= value].mean() < 0.02:
                rules.append((column, '>', value))
                break
    return rules

rules = extract_rules(X_train, y_train)

def apply_rules(X, rules):
    correct = np.zeros(len(X))
    for rule in rules:
        column, operator, value = rule
        if operator == '<=':
            correct[X[column] > value] += 1
        if operator == '>':
            correct[X[column] <= value] += 1
    
    correct /= len(rules)
    pred = (correct > 0.5).astype(int)
    return pred

predictions = apply_rules(X_train, rules)
accuracy = (predictions == y_train).mean()
print(f'Number of rules: {len(rules)}')
print(f'Accuracy: {accuracy:.2f}')

Number of rules: 4
Accuracy: 0.65


I separate the dataset if more than 98% of one class has value greater than or smaller than the given value. The main strategy is using this 98% to try and filter out which rule is the best. In the end, I had 4 rules and an accuracy of 65%.

(b) Use the algorithms developed in (a) on different datasets. Again, observe how your choices make a difference.

**Solution**: 

In [62]:
# Load the Iris dataset
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = (data.target == 3).astype(int)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df[data.feature_names], df['target'], test_size=0.2, random_state=42)
rules = extract_rules(X_train, y_train)
predictions = apply_rules(X_train, rules)
accuracy = (predictions == y_train).mean()
print(f'Number of rules: {len(rules)}')
print(f'Accuracy: {accuracy:.2f}')

Number of rules: 4
Accuracy: 1.00


Given different datasets, the results are different. Some might be more easily separable, such as using a different type of flower in this case. Still using 4 rules, the accuracy becomes 100%. 

(c) Finally, use the programs developed in (a) on a completely random dataset, generated artificially. Vary your strategies but also the number of input columns as well as the number of instances. How many if-then clauses do you need?

**Solution**: 

In [67]:
print("------2 columns, 100 instances-----")
X, y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_classes=2)
X = pd.DataFrame(X, columns=['x1', 'x2'])

rules = extract_rules(X, y)
predictions = apply_rules(X, rules)
accuracy = (predictions == y).mean()
print(f'Number of rules: {len(rules)}')
print(f'Accuracy: {accuracy:.2f}')

print("------10 columns, 500 instances-----")
X, y = make_classification(n_samples=500, n_features=10, n_informative=10, n_redundant=0, n_classes=2)
X = pd.DataFrame(X, columns=[f'x{i}' for i in range(10)])

rules = extract_rules(X, y)
predictions = apply_rules(X, rules)
accuracy = (predictions == y).mean()
print(f'Number of rules: {len(rules)}')
print(f'Accuracy: {accuracy:.2f}')

------2 columns, 100 instances-----
Number of rules: 2
Accuracy: 0.49
------10 columns, 500 instances-----
Number of rules: 10
Accuracy: 0.49


On generated artificial datasets, the accuracy is only around 50%. The number of rules I used is equivalent to the number of input columns.

# Exercise 3: Compression

(a) Create a long random string using a Python program, and use a lossless compression algorithm of your choice to compress the string. Note the compression ratio.

In [22]:
import zlib
import random
import string

string_length = 1000
original_string = ''.join(random.choice(string.ascii_letters) for _ in range(string_length))
original_string = original_string.encode('ascii')
compressed_string = zlib.compress(original_string)

print(f'Original string: {len(original_string)} bytes')
print(f'Compressed string: {len(compressed_string)} bytes')

compression_ratio = len(original_string) / len(compressed_string)
print(f'Compression ratio: {compression_ratio:.2f}')

Original string: 1000 bytes
Compressed string: 748 bytes
Compression ratio: 1.34


(b) What is the expected compression ratio in (a)? Explain why?

The expected compression ratio should be 1 because if the string is truely random, you cannot find any patterns to compress down the information.